In [111]:
import pandas as pd
from pathlib import Path
import re

### Scenario delta
This scenario calculates the differences between the policy and BAU scenarios.

In [112]:
# Inputs and outputs
results_path = Path("../results")
tables_path = Path("tables")

In [113]:
# Load case information
cases = [
    "baseline_2019", "today_2024", "future_2030",
    "policy_co2_100EUR", "policy_co2_200EUR", "policy_co2_500EUR", "policy_co2_750EUR","policy_co2_1000EUR", "policy_co2_1500EUR", "policy_co2_2000EUR",
    "policy_thermic_20pct", "policy_thermic_50pct", "policy_thermic_75pct", "policy_thermic_100pct",
    "policy_elez", "policy_electric", "policy_cost_parity" #, "policy_cost_parity_high_range"
]

df_cases = []

for case in cases:
    df_partial = pd.read_parquet(results_path / "scenario_solutions/{}.vehicles.parquet".format(case))
    f_electric = df_partial["vehicle_type"].str.endswith("electric")

    df_cases.append({
        "case": case,

        "vehicles": len(df_partial[["vehicle_id", "depot", "operator"]].drop_duplicates()),
        "thermic_vehicles": len(df_partial[~f_electric][["vehicle_id", "depot", "operator"]].drop_duplicates()),
        "electric_vehicles": len(df_partial[f_electric][["vehicle_id", "depot", "operator"]].drop_duplicates()),
        
        "distance_km": df_partial["distance_m"].sum() * 1e-3,
        "thermic_distance_km": df_partial[~f_electric]["distance_m"].sum() * 1e-3,
        "electric_distance_km": df_partial[f_electric]["distance_m"].sum() * 1e-3,

        "fuel_L": df_partial["fuel_L"].sum(),
        "electricity_kWh": df_partial["electricity_Wh"].sum() * 1e-3,
        "energy_kWh": df_partial["energy_Wh"].sum() * 1e-3,
        "co2eq_kg": df_partial["co2eq_g"].sum() * 1e-3,

        "total_cost_EUR": df_partial["total_cost_EUR"].sum(),

        "deliveries": df_partial["deliveries"].sum()
    })

df_cases = pd.DataFrame.from_records(df_cases)
df_cases["cost_per_delivery_EUR"] = df_cases["total_cost_EUR"] / df_cases["deliveries"]
df_cases["cost_per_distance_EUR_km"] = df_cases["total_cost_EUR"] / df_cases["distance_km"]

df_cases = df_cases.drop(columns = [
    "deliveries"
])

In [114]:
# Calculate offset with reference cases
reference_cases = ["baseline_2019", "today_2024"]
ignore_cases = []
df_comparison = []

comparison_columns = ["distance_km", "energy_kWh", "co2eq_kg", "cost_per_delivery_EUR"]

df_comparison = df_cases[~df_cases["case"].isin(reference_cases) & ~df_cases["case"].isin(ignore_cases)]
df_comparison = df_comparison[["case"] + comparison_columns].copy()

df_plot = []

for reference_case in reference_cases:
    year = int(reference_case.split("_")[-1])

    df_partial = df_cases[~df_cases["case"].isin(reference_cases) & ~df_cases["case"].isin(ignore_cases)]
    df_partial = df_partial[["case"] + comparison_columns].copy()

    for column in comparison_columns:
        reference_value = df_cases.loc[df_cases["case"] == reference_case, column].values[0]
        df_comparison["{}_{}".format(column, year)] = 1e2 * (df_comparison[column] - reference_value) / reference_value
        df_partial[column] = 1e2 * (df_partial[column] - reference_value) / reference_value

    df_partial["reference"] = year
    df_plot.append(df_partial)

df_plot = pd.concat(df_plot)

years = [2024, 2019]

df_comparison = df_comparison[["case"] + [
    "{}_{}".format(column, year)
    for year in years for column in comparison_columns
]]

df_comparison.columns = pd.MultiIndex.from_tuples([("case", "")] + [
    [year, column.split("_")[0]]
    for year in years for column in comparison_columns
])

df_comparison.round(2)

case     2024                         2019                \
                          distance energy  co2eq  cost distance energy  co2eq   
2             future_2030    26.31  46.13  50.51 -6.94    50.78  86.45  96.08   
3       policy_co2_100EUR    26.94  43.76  47.14 -6.39    51.54  83.41  91.69   
4       policy_co2_200EUR    26.84  39.70  41.96 -5.78    51.42  78.23  84.94   
5       policy_co2_500EUR    29.71  30.27  29.31 -4.34    54.85  66.21  68.45   
6       policy_co2_750EUR    30.46  18.78  11.96 -3.14    55.74  51.55  45.86   
7      policy_co2_1000EUR    30.68   6.42  -6.31 -2.21    56.01  35.78  22.05   
8      policy_co2_1500EUR    31.33 -13.05 -35.46 -0.73    56.78  10.93 -15.92   
9      policy_co2_2000EUR    32.21 -24.43 -51.84  0.24    57.83  -3.58 -37.26   
10   policy_thermic_20pct    29.61  30.29  29.09 -5.10    54.72  66.23  68.17   
11   policy_thermic_50pct    37.42  20.06  14.46 -3.35    64.05  53.18  49.11   
12   policy_thermic_75pct    56.30   0.22 -15.52 -2.05    86.59  27.87  10.06   
13  policy_thermic_100pct    80.38 -36.76 -71.96 -1.47   115.34 -19.31 -63.47   
14            policy_elez    42.34   7.97  -6.17 -2.62    69.92  37.75  22.23   
15        policy_electric    79.57 -42.10 -79.49 -1.41   114.37 -26.13 -73.29   
16     policy_cost_parity    25.66 -42.91 -79.78 -8.63    50.02 -27.16 -73.66   

           
     cost  
2  -12.23  
3  -11.72  
4  -11.14  
5   -9.78  
6   -8.65  
7   -7.77  
8   -6.37  
9   -5.46  
10 -10.50  
11  -8.84  
12  -7.62  
13  -7.08  
14  -8.16  
15  -7.02  
16 -13.82

In [115]:
# Prepare table with deltas
df_table = df_comparison.copy()

def replace_variable(variable):
    if variable == "distance": return "Dist."
    if variable == "co2eq": return "Emissions"
    if variable == "energy": return "Energy"
    if variable == "cost": return "Cost"
    return variable

df_table.columns = pd.MultiIndex.from_tuples([("case", "")] + [
    ["$\\Delta {}$".format(year), replace_variable(column.split("_")[0])]
    for year in years for column in comparison_columns
])

df_table = df_table.set_index("case")
df_table.index.name = "[\\%]"

def replace_name(name):
    if name == "baseline_2019": return "Baseline 2019"
    if name == "today_2024": return "Today 2024"
    if name == "future_2030": return "Future 2030"
    if name == "policy_elez": return "Electric LEZ"
    if name == "policy_electric": return "100\\% Electric"
    if name == "policy_cost_parity": return "Cost parity"

    if name.startswith("policy_co2"):
        return "Carbon tax ({} EUR)".format(name.split("_")[-1].replace("EUR", ""))

    if name.startswith("policy_thermic"):
        return "ICV tax ({}\\%)".format(name.split("_")[-1].replace("pct", ""))

    return name

df_table.index = [replace_name(name) for name in df_table.index]

with open(tables_path / "delta.tex", "w+") as f:
    table = df_table.to_latex(
        caption = "Changes in key indicators compared to reference scenarios [\\%]", 
        label = "tab:delta", sparsify = True,
        float_format = lambda f: "{:.2f}".format(f)
    )

    table = table.replace("Carbon tax (100 EUR)", "\\midrule\nCarbon tax (100 EUR)")
    table = table.replace("ICV tax (20\\%)", "\\midrule\nICV tax (20\\%)")
    table = table.replace("Electric LEZ", "\\midrule\nElectric LEZ")

    table = table.replace("\\begin{table}", "\\begin{table}\n\\centering")
    
    table = table.replace("lrrrrrrrr", "lrrrr|rrrr")
    table = re.sub(r"(-[0-9.]+)", "\\\\textbf{\\1}", table)
    table = table.replace("{r}", "{c}")
    table = table.replace("{$\\Delta 2019$} \\\\", "{$\\Delta 2019$} \\\\ \n \\midrule")

    f.write(table)

df_table

$\Delta 2024$                                  \
                              Dist.     Energy  Emissions      Cost   
Future 2030               26.305460  46.134302  50.514445 -6.940151   
Carbon tax (100 EUR)      26.941178  43.755183  47.144032 -6.392537   
Carbon tax (200 EUR)      26.837170  39.695939  41.959699 -5.775972   
Carbon tax (500 EUR)      29.714323  30.272030  29.305426 -4.338471   
Carbon tax (750 EUR)      30.456727  18.782486  11.960671 -3.140697   
Carbon tax (1000 EUR)     30.683541   6.418037  -6.311802 -2.209611   
Carbon tax (1500 EUR)     31.326449 -13.051597 -35.459775 -0.727543   
Carbon tax (2000 EUR)     32.205335 -24.428294 -51.836584  0.244873   
ICV tax (20\%)            29.606634  30.290446  29.088946 -5.099497   
ICV tax (50\%)            37.418410  20.061937  14.456167 -3.345217   
ICV tax (75\%)            56.299936   0.222079 -15.515512 -2.053756   
ICV tax (100\%)           80.384306 -36.757863 -71.962674 -1.473306   
Electric LEZ              42.338810   7.969738  -6.170972 -2.623651   
100\% Electric            79.567938 -42.103903 -79.493877 -1.414389   
Cost parity               25.664802 -42.907264 -79.778418 -8.626734   

                      $\Delta 2019$                                   
                              Dist.     Energy  Emissions       Cost  
Future 2030               50.783168  86.447802  96.080770 -12.233439  
Carbon tax (100 EUR)      51.542085  83.412365  91.690008 -11.716973  
Carbon tax (200 EUR)      51.417921  78.233313  84.936185 -11.135479  
Carbon tax (500 EUR)      54.852660  66.209667  68.450992  -9.779743  
Carbon tax (750 EUR)      55.738940  51.550548  45.855334  -8.650099  
Carbon tax (1000 EUR)     56.009710  35.775167  22.051103  -7.771973  
Carbon tax (1500 EUR)     56.777212  10.934521 -15.921046  -6.374206  
Carbon tax (2000 EUR)     57.826424  -3.580622 -37.255725  -5.457102  
ICV tax (20\%)            54.724100  66.233164  68.168976 -10.497482  
ICV tax (50\%)            64.049781  53.182956  49.106310  -8.842986  
ICV tax (75\%)            86.590504  27.869954  10.061088  -7.624984  
ICV tax (100\%)          115.342368 -19.311502 -63.474732  -7.077550  
Electric LEZ              69.923743  37.754929  22.234567  -8.162463  
100\% Electric           114.367791 -26.132333 -73.285910  -7.021984  
Cost parity               50.018351 -27.157314 -73.656592 -13.824088

In [119]:
df_table = df_cases.copy()

df_2019 = df_table[df_table["case"] == "baseline_2019"]
df_2024 = df_table[df_table["case"] == "today_2024"]

indicators = ["distance_km", "energy_kWh", "co2eq_kg", "total_cost_EUR", "cost_per_delivery_EUR", "cost_per_distance_EUR_km"]

df_table = df_table[["case"] + indicators]
# df_table = df_table[~df_table["case"].isin(["baseline_2019", "today_2024"])]

for column in indicators:
    df_table[column + "_2024"] = df_2024[column].values[0]
    df_table[column + "_2019"] = df_2019[column].values[0]

    df_table[column + "_2024"] = (df_table[column] - df_table[column + "_2024"]) / df_table[column + "_2024"] * 100
    df_table[column + "_2019"] = (df_table[column] - df_table[column + "_2019"]) / df_table[column + "_2019"] * 100

for column in indicators:
    #df_table[column] = df_table[column].round(0).astype(int)
    df_table[column + "_2019"] = df_table[column + "_2019"].round(0).astype(int)
    df_table[column + "_2024"] = df_table[column + "_2024"].round(0).astype(int)

df_table = df_table.set_index("case")

columns = []
for c in indicators:
    columns.append(c)
    #columns.append(c + "_2024")
    #columns.append(c + "_2019")

df_table = df_table[columns]

names = {
    "distance_km": "Dist. [km]",
    "energy_kWh": "Energy [kWh]",
    "co2eq_kg": "Emis. [kgCO2eq]",
    "total_cost_EUR": "Cost [EUR]",
    "cost_per_delivery_EUR": "Cost [EUR/par]",
    "cost_per_distance_EUR_km": "Cost [EUR/km]",
}

df_table.columns = [
    "\\tiny $\\Delta 2019$" if "2019" in c else
    "\\tiny $\\Delta 2024$" if "2024" in c else
    names[c]
    for c in df_table.columns
]

df_table


df_table.index = [replace_name(name) for name in df_table.index]

df_table = df_table.style.format(thousands = ",", precision = 0) # .hide(axis = "index")


selector = (
    df_table.index, 
    [c for c in df_table.columns if "EUR/" in c]
)

df_table = df_table.format(thousands = ",", precision = 2, subset = selector) # .hide(axis = "index")


with open(tables_path / "indicators.tex", "w+") as f:
    table = df_table.to_latex(
        caption = "System indicators for the policy scenarios", 
        label = "tab:indicators", hrules = True # sparsify = True,
        # float_format = lambda f: "{:.2f}".format(f)
    )

    table = table.replace("Carbon tax (100 EUR)", "\\midrule\nCarbon tax (100 EUR)")
    table = table.replace("ICV tax (20\\%)", "\\midrule\nICV tax (20\\%)")
    table = table.replace("Electric LEZ", "\\midrule\nElectric LEZ")

    table = table.replace("\\begin{table}", "\\begin{table}\n\\centering")
    
    #table = table.replace("lrrrrrrrr", "lrrrr|rrrr")
    #table = re.sub(r"(-[0-9.]+)", "\\\\textbf{\\1}", table)
    #table = table.replace("{r}", "{c}")
    #table = table.replace("{$\\Delta 2019$} \\\\", "{$\\Delta 2019$} \\\\ \n \\midrule")

    f.write("\\begin{landscape}\n")
    f.write(table)
    f.write("\\end{landscape}")

df_table

,Dist. [km],Energy [kWh],Emis. [kgCO2eq],Cost [EUR],Cost [EUR/par],Cost [EUR/km]
Baseline 2019,"16,571","8,929","2,222","41,438",1.54,2.50
Today 2024,"19,783","11,392","2,895","52,119",1.45,2.63
Future 2030,"24,987","16,647","4,357","73,492",1.35,2.94
Carbon tax (100 EUR),"25,113","16,376","4,259","73,924",1.36,2.94
Carbon tax (200 EUR),"25,092","15,914","4,109","74,411",1.36,2.97
Carbon tax (500 EUR),"25,661","14,840","3,743","75,546",1.39,2.94
Carbon tax (750 EUR),"25,808","13,531","3,241","76,492",1.40,2.96
Carbon tax (1000 EUR),"25,853","12,123","2,712","77,227",1.42,2.99
Carbon tax (1500 EUR),"25,980","9,905","1,868","78,398",1.44,3.02
Carbon tax (2000 EUR),"26,154","8,609","1,394","79,166",1.45,3.03
